In [ ]:
!pip install pandarallel
!pip install gensim --upgrade
!pip install pyLDAvis --upgrade
!pip install pyarrow
!pip install vaderSentiment
!pip install tqdm
!pip install spacy
!pip install ipywidgets

!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.1 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=720ffa7e533b0ac84d10d5c572331996e9d0413f05d2b7f9c1b2a0e4e1b34b47
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 101.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import nltk
import nltk.corpus
from nltk.text import Text
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.metrics import jaccard_distance
from nltk.text import Text
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import os
import requests
import pprint
from joblib import dump, load
from pandarallel import pandarallel
import multiprocessing
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import spacy
from spacy import displacy
from spacy.util import minibatch, compounding
import time
from collections import Counter
from pprint import pprint


import os
import warnings
warnings.simplefilter('ignore')

import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

nltk.download('all')

spacy.__version__

Mounted at /content/drive


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

'3.7.5'

In [ ]:
pandarallel.initialize(nb_workers = multiprocessing.cpu_count()-1, use_memory_fs=False, progress_bar = True)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/df_sentiment_analysis.parquet', engine = 'pyarrow')
df

,url,date,language,title,text,year,month,day,dayofweek,title_cleaned,text_cleaned,title_cleaned_tokens,text_cleaned_tokens,initial_sentiment,sentiment_score,sentiment
0,http://www.dataweek.co.za/21690r,2024-04-05,en,Flash for AI - 28 March 2024 - EBV Electrolink - Dataweek,"\nFlash for AI - 28 March 2024 - EBV Electrolink - Dataweek\nHome\nAbout us\nBack issues / E-book / PDF\nSubscribe\nAdvertise\n\n \n\nEMP Handbook\n\nCategories\n\n▸ Editor's Choice\n▸ Multimedia, Videos\n▸ AI & ML\n▸ Analogue, Mixed Signal, LSI\n▸ Circuit & System Protection\n▸ Computer/Embedded Technology\n▸ Design Automation\n▸ DSP, Micros & Memory\n▸ Edge Computing & IIoT\n▸ Electronics Technology\n▸ Enclosures, Racks, Cabinets & Panel Products\n▸ Events\n▸ Interconnection\n▸ Manufacturi...",2024,4,5,4,Flash for AI 28 March 2024 EBV Electrolink Dataweek,Flash for AI 28 March 2024 EBV Electrolink Dataweek Home About us Back issues Ebook PDF Subscribe Advertise EMP Handbook Categories Editors Choice Multimedia Videos AI ML Analogue Mixed Signal LSI Circuit System Protection ComputerEmbedded Technology Design Automation DSP Micros Memory Edge Computing IIoT Electronics Technology Enclosures Racks Cabinets Panel Products Events Interconnection Manufacturing Production Technology Hardware Services News OptoElec...,"[flash, ai, march, ebv, electrolink, dataweek]","[flash, ai, march, ebv, electrolink, dataweek, home, u, back, issue, ebook, pdf, subscribe, advertise, emp, handbook, category, editor, choice, multimedia, video, ai, ml, analogue, mixed, signal, lsi, circuit, system, protection, computerembedded, technology, design, automation, dsp, micros, memory, edge, computing, iiot, electronics, technology, enclosure, rack, cabinet, panel, product, event, interconnection, manufacturing, production, technology, hardware, service, news, optoelectronics, ...",0,0.9995,extremely positive
1,http://www.mysmartrend.com/news-briefs/technical-analysis/cr-bard-inc-has-returned-489-smartrend-recommendation-bcr-2,2020-04-17,en,Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend,"\n\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend\n \nOWL LOGIN / \n ADVISOR LOGIN\n\nHome\nAdvantages\nProducts\n\nNews &amp Analysis\n\n\nResources\nTestimonials\n\n *Supporting investors at home - 60% off SmarTrend Products - Coupon Code: COVID19 \nReturn to Headlines\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) \nWritten on Fri, 04/17/2020 - 1:08am\n By Shiri Gupta\nSmarTrend identified an Uptrend for Cr Ba...",2020,4,17,4,Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Comtex SmarTrend,Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Comtex SmarTrend OWL LOGIN ADVISOR LOGIN Home Advantages amp Analysis Resources Testimonials Supporting investors at home 60 off SmarTrend Products Coupon Code COVID19 Return to Headlines Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Written on Fri 04172020 108am By Shiri Gupta SmarTrend identified an Uptrend for Cr Bard Inc BCR on December 23rd 2016 at 22245 In approximate...,"[cr, bard, inc, returned, since, smartrend, recommendation, bcr, comtex, smartrend]","[cr, bard, inc, returned, since, smartrend, recommendation, bcr, comtex, smartrend, owl, login, advisor, login, home, advantage, amp, analysis, resource, testimonial, supporting, investor, home, smartrend, product, coupon, code, covid19, return, headline, cr, bard, inc, returned, since, smartrend, recommendation, bcr, written, fri, 108am, shiri, gupta, smartrend, identified, uptrend, cr, bard, inc, bcr, december, 23rd, approximately, month, cr, bard, inc, returned, today, recent, price, 3312...",1,0.9821,extremely positive
2,https://abcnews.go.com/Entertainment/video/ai-hiring-firing-focus-book-journalist-hilke-schellmann-108912618,2024-04-06,en,Video AI used for hiring and firing focus of book by journalist Hilke Schellmann - ABC News,\n\nVideo AI used for h

In [ ]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
df_dictionary = corpora.Dictionary(df['text_cleaned_tokens'])

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
df_doc_term_matrix = [df_dictionary.doc2bow(doc) for doc in df['text_cleaned_tokens']]

In [ ]:
df_lda_model = LdaMulticore(corpus=df_doc_term_matrix,
                   id2word=df_dictionary,
                   num_topics=5,
                   random_state=142,
                   passes=10,
                   eta='auto',
                   workers=multiprocessing.cpu_count()-1)

# Print the Keyword in the 5 topics
pprint(df_lda_model.print_topics())
doc_lda = df_lda_model[df_doc_term_matrix]

[(0,
  '0.009*"share" + 0.008*"business" + 0.007*"service" + 0.007*"public" + '
  '0.006*"ai" + 0.006*"technology" + 0.006*"product" + 0.006*"medium" + '
  '0.006*"price" + 0.005*"policy"'),
 (1,
  '0.021*"ai" + 0.009*"data" + 0.006*"technology" + 0.005*"business" + '
  '0.005*"new" + 0.004*"model" + 0.004*"u" + 0.004*"intelligence" + '
  '0.004*"company" + 0.003*"use"'),
 (2,
  '0.022*"ai" + 0.007*"gray" + 0.006*"medium" + 0.006*"group" + '
  '0.006*"technology" + 0.005*"data" + 0.005*"company" + 0.005*"solution" + '
  '0.005*"customer" + 0.005*"platform"'),
 (3,
  '0.030*"market" + 0.014*"ai" + 0.010*"stock" + 0.008*"report" + '
  '0.008*"company" + 0.007*"global" + 0.007*"intelligence" + 0.006*"growth" + '
  '0.006*"artificial" + 0.006*"share"'),
 (4,
  '0.011*"ai" + 0.010*"u" + 0.007*"new" + 0.005*"said" + 0.005*"video" + '
  '0.005*"ago" + 0.003*"story" + 0.003*"chatgpt" + 0.003*"google" + '
  '0.003*"hour"')]


In [ ]:
%%time

df_lda_display = gensimvis.prepare(df_lda_model, df_doc_term_matrix, df_dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(df_lda_display)

CPU times: user 2min 6s, sys: 3.46 s, total: 2min 10s
Wall time: 2min 24s


In [ ]:
pos_df = df[df["sentiment"].isin(["extremely positive", "positive"])]
neg_df = df[df["sentiment"].isin(["extremely negative", "negative"])]

In [ ]:
# Creating the term dictionary of positive courpus, where every unique term is assigned an index.
pos_df_dictionary = corpora.Dictionary(pos_df['text_cleaned_tokens'])

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
pos_df_doc_term_matrix = [pos_df_dictionary.doc2bow(doc) for doc in pos_df['text_cleaned_tokens']]

In [ ]:
pos_df_lda_model = LdaMulticore(corpus=pos_df_doc_term_matrix,
                   id2word=pos_df_dictionary,
                   num_topics=5,
                   random_state=142,
                   passes=10,
                   eta='auto',
                   workers=multiprocessing.cpu_count()-1)

# Print the Keyword in the 5 topics
pprint(pos_df_lda_model.print_topics())
doc_lda = pos_df_lda_model[pos_df_doc_term_matrix]

[(0,
  '0.028*"market" + 0.013*"ai" + 0.012*"share" + 0.009*"stock" + '
  '0.008*"report" + 0.007*"price" + 0.007*"global" + 0.007*"intelligence" + '
  '0.006*"artificial" + 0.006*"company"'),
 (1,
  '0.013*"u" + 0.012*"ai" + 0.006*"new" + 0.004*"said" + 0.004*"technology" + '
  '0.004*"world" + 0.003*"use" + 0.003*"intelligence" + 0.003*"artificial" + '
  '0.003*"email"'),
 (2,
  '0.007*"public" + 0.007*"ai" + 0.007*"ago" + 0.006*"business" + '
  '0.005*"service" + 0.005*"u" + 0.005*"technology" + 0.005*"medium" + '
  '0.005*"weather" + 0.004*"contact"'),
 (3,
  '0.021*"ai" + 0.007*"new" + 0.005*"company" + 0.005*"google" + 0.004*"model" '
  '+ 0.004*"best" + 0.004*"data" + 0.004*"chatgpt" + 0.004*"u" + 0.004*"tech"'),
 (4,
  '0.022*"ai" + 0.009*"data" + 0.007*"technology" + 0.006*"gray" + '
  '0.005*"medium" + 0.005*"company" + 0.005*"solution" + 0.005*"group" + '
  '0.005*"customer" + 0.005*"business"')]


In [ ]:
%%time

pos_df_lda_display = gensimvis.prepare(pos_df_lda_model, pos_df_doc_term_matrix, pos_df_dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(pos_df_lda_display)

CPU times: user 1min 57s, sys: 4.04 s, total: 2min 1s
Wall time: 2min 14s


In [ ]:
# Creating the term dictionary of the negative courpus, where every unique term is assigned an index.
neg_df_dictionary = corpora.Dictionary(neg_df['text_cleaned_tokens'])

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
neg_df_doc_term_matrix = [neg_df_dictionary.doc2bow(doc) for doc in neg_df['text_cleaned_tokens']]

In [ ]:
neg_df_lda_model = LdaMulticore(corpus=neg_df_doc_term_matrix,
                   id2word=neg_df_dictionary,
                   num_topics=5,
                   random_state=142,
                   passes=10,
                   eta='auto',
                   workers=multiprocessing.cpu_count()-1)

# Print the Keyword in the 5 topics
pprint(neg_df_lda_model.print_topics())
doc_lda = neg_df_lda_model[neg_df_doc_term_matrix]

[(0,
  '0.016*"ai" + 0.005*"data" + 0.005*"company" + 0.004*"said" + 0.004*"u" + '
  '0.004*"new" + 0.003*"google" + 0.003*"technology" + 0.003*"icon" + '
  '0.003*"chatgpt"'),
 (1,
  '0.007*"ai" + 0.007*"u" + 0.005*"said" + 0.004*"image" + 0.004*"new" + '
  '0.004*"medium" + 0.003*"trump" + 0.003*"say" + 0.003*"video" + '
  '0.003*"people"'),
 (2,
  '0.013*"ai" + 0.006*"said" + 0.005*"new" + 0.004*"u" + 0.003*"technology" + '
  '0.003*"one" + 0.003*"could" + 0.003*"use" + 0.003*"people" + 0.002*"say"'),
 (3,
  '0.009*"ai" + 0.008*"u" + 0.004*"new" + 0.004*"cancer" + 0.004*"india" + '
  '0.003*"health" + 0.003*"technology" + 0.003*"world" + 0.003*"business" + '
  '0.003*"said"'),
 (4,
  '0.015*"ago" + 0.008*"hour" + 0.008*"video" + 0.007*"ai" + 0.006*"story" + '
  '0.006*"new" + 0.006*"u" + 0.006*"weather" + 0.005*"said" + 0.005*"day"')]


In [ ]:
%%time

neg_df_lda_display = gensimvis.prepare(neg_df_lda_model, neg_df_doc_term_matrix, neg_df_dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(neg_df_lda_display)

CPU times: user 11.6 s, sys: 2.17 s, total: 13.8 s
Wall time: 16.8 s
